In [4]:
import pandas as pd
from Functions import *

In [5]:
data_tpts, data_x, data_y = data_deal()
MISE, index_z = compared(n = 10, seed_z = 1, data_x = data_x, data_y = data_y, data_tpts = data_tpts, 
                   n_basis = 4, order = 3, test_size = 0.2, plot_flag=True,
                   method_list = ['lin_b', 'knn_b', 'gbdt_b', 'xgb_b', 'MBR_XGB'])

Data processing completed!


In [10]:
df = pd.DataFrame(index_z)
# 保留第一列两位小数
df['R2_mean'] = df['R2_mean'].round(3)
df['R2_var'] = np.sqrt(df['R2_var']).round(3)
df['Mise_mean'] = df['Mise_mean'].round(3)
df['Mise_var'] = np.sqrt(df['Mise_var']).round(3)
df['Rmse_mean'] = df['Rmse_mean'].round(3)
df['Rmse_var'] = np.sqrt(df['Rmse_var']).round(3)
print(df)
# df.to_csv('results.csv', index=False)

  method_name  R2_mean  R2_var  Mise_mean  Mise_var  Rmse_mean  Rmse_var
0       lin_b    0.860   0.134      0.499     0.520      0.122     0.068
1       knn_b    0.890   0.106      0.401     0.437      0.110     0.067
2      gbdt_b    0.901   0.104      0.371     0.418      0.097     0.071
3       xgb_b    0.897   0.106      0.379     0.413      0.100     0.069
4     MBR_XGB    0.905   0.106      0.357     0.418      0.095     0.070
